# Google Common Expression Language (CEL) in Python

See https://github.com/google/cel-spec

See https://github.com/cloud-custodian/cloud-custodian/issues/5759 for the Cloud Custodian rationale for including CEL as a replacement for the filter language.

## Goal

Here's the target state.

```
policies:
   - name: compute-check
      resource: gcp.instance
      filters:
        - type: cel
           expr: |
               resource.creationTimestamp < timestamp("2018-08-03T16:00:00-07:00") &&
               resource.deleteProtection == false &&
               ((resource.name.startsWith(
                   "projects/project-123/zones/us-east1-b/instances/dev") ||
               (resource.name.startsWith(
                   "projects/project-123/zones/us-east1-b/instances/prod"))) &&
               resource.instanceSize == "m1.standard")
```

We've replaced a legacy YAML filter expression with a CEL expression using easier-to-read logic and comparison operators. 

Also. We've provided an object, `resource`, to the CEL engine. This has the attributes we're interested in, converted into a friendly CEL data structures (e.g., the timestamps.)

## Implementation

There are several steps to creating and evaluating a CEL expression.

1. Create an environment.
2. Parse the expression.
3. Build a "program" from the expression and any additional functions required.
4. Evaluate the program with the variable bindings.

In [1]:
import logging
logging.getLogger('').setLevel(logging.WARNING)

In [2]:
import celpy

In [3]:
env = celpy.Environment()

In [4]:
CEL = """
resource.creationTimestamp < timestamp("2018-08-03T16:00:00-07:00") &&
resource.deleteProtection == false &&
((resource.name.startsWith(
   "projects/project-123/zones/us-east1-b/instances/dev") ||
(resource.name.startsWith(
   "projects/project-123/zones/us-east1-b/instances/prod"))) &&
resource.instanceSize == "m1.standard")
"""

In [5]:
ast = env.compile(CEL)

In [6]:
functions = {}
prgm = env.program(ast, functions)

In [7]:
activation = {
    "resource": 
         celpy.celtypes.MapType({
            "creationTimestamp": celpy.celtypes.TimestampType("2018-07-06T05:04:03Z"),
            "deleteProtection": celpy.celtypes.BoolType(False),
            "name": celpy.celtypes.StringType("projects/project-123/zones/us-east1-b/instances/dev/ec2"),
            "instanceSize": celpy.celtypes.StringType("m1.standard"),
             # MORE WOULD GO HERE
        })
}

In [8]:
prgm.evaluate(activation)

BoolType(True)

## JSON Conversion

We have a handy JSON -> CEL function available. The subtlety is that it doesn't know what's supposed to be a timestamp.

In [9]:
import json
document = json.loads(
"""
{
    "creationTimestamp": "2018-07-06T05:04:03Z",
    "deleteProtection": false,
    "name": "projects/project-123/zones/us-east1-b/instances/dev/ec2",
    "instanceSize": "m1.standard"
}
"""
)

resource = celpy.json_to_cel(document)
resource

MapType({StringType('creationTimestamp'): StringType('2018-07-06T05:04:03Z'), StringType('deleteProtection'): BoolType(False), StringType('name'): StringType('projects/project-123/zones/us-east1-b/instances/dev/ec2'), StringType('instanceSize'): StringType('m1.standard')})

In [10]:

prgm.evaluate({'resource': resource})

CELEvalError: ("found no matching overload for 'relation_lt' applied to '(<class 'celpy.celtypes.StringType'>, <class 'celpy.celtypes.TimestampType'>)'", <class 'TypeError'>, ("'<' not supported between instances of 'StringType' and 'TimestampType'",))

What can we do?

We have some choices:

- Conversion in CEL

- Conversion of the source document before CEL evaluation.

## Conversion in CEL

We can convert input strings to more useful CEL types explicitly. 

Instead of `resource.creationTimestamp`, we use `timestamp(resource.creationTimestamp)`. 

In [39]:
CEL2 = """
timestamp(resource.creationTimestamp) < timestamp("2018-08-03T16:00:00-07:00") &&
resource.deleteProtection == false &&
((resource.name.startsWith(
   "projects/project-123/zones/us-east1-b/instances/dev") ||
(resource.name.startsWith(
   "projects/project-123/zones/us-east1-b/instances/prod"))) &&
resource.instanceSize == "m1.standard")
"""
ast2 = env.compile(CEL2)
prgm2 = env.program(ast2, functions)

In [40]:
prgm2.evaluate({'resource': resource})

BoolType(True)

## Conversion of the source document CEL

This requires pre-processing to convert timestamps before CEL evaluation. This (in turn) requires a careful definition of the source schema for the JSON in order to perform the conversions.

This seems fraught with potential complexities.

## Visibility

We can enable logging. In a notebook, we have to be careful because the log lines will go to the notebook log if we're not careful. We want to have our own handlers to capture the output in a separate file.

And, yes, this is **verbose**. Suggestions are welcome.

In [41]:
import logging
logging.basicConfig()
logging.getLogger('').setLevel(logging.INFO)

In [42]:
prgm.evaluate({'resource': resource})

INFO:Evaluator:activation: Activation(annotations={}, package=None, vars={'resource': MapType({StringType('creationTimestamp'): StringType('2018-07-06T05:04:03Z'), StringType('deleteProtection'): BoolType(False), StringType('name'): StringType('projects/project-123/zones/us-east1-b/instances/dev/ec2'), StringType('instanceSize'): StringType('m1.standard')})})
INFO:Evaluator:functions: ChainMap({}, {'!_': <function logical_not at 0x7ff8c76fc8c0>, '-_': <built-in function neg>, '_+_': <built-in function add>, '_-_': <built-in function sub>, '_*_': <built-in function mul>, '_/_': <built-in function truediv>, '_%_': <built-in function mod>, '_<_': <function lt at 0x7ff8c7767f80>, '_<=_': <function le at 0x7ff8c776f050>, '_>=_': <function ge at 0x7ff8c776f0e0>, '_>_': <function gt at 0x7ff8c776f170>, '_==_': <function eq at 0x7ff8c776f200>, '_!=_': <function ne at 0x7ff8c776f290>, '_in_': <function operator_in at 0x7ff8c7767e60>, '_||_': <function logical_or at 0x7ff8c76fc950>, '_&&_': <fun

CELEvalError: ("found no matching overload for 'relation_lt' applied to '(<class 'celpy.celtypes.StringType'>, <class 'celpy.celtypes.TimestampType'>)'", <class 'TypeError'>, ("'<' not supported between instances of 'StringType' and 'TimestampType'",))